# Install Dependencies
Install the required Python packages to run LangChain and interact with LLMs. This ensures all modules are available before building the app.

In [1]:
from IPython.core.debugger import prompt
!pip install -qU "langchain[google-genai]"

# Set Google API Key
Load the Google API key from the environment to authenticate our requests. This protects the key from being hard-coded into the notebook.

In [3]:
import os
from dotenv import load_dotenv
import getpass

load_dotenv()

google_api_key = os.environ.get("GOOGLE_API_KEY")

if not google_api_key:
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

# Create an LLM
Initialize the Gemini language model using LangChain’s wrapper. This model will generate text based on the prompts we define.


In [7]:
from langchain.chat_models import init_chat_model


In [8]:
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [9]:
model.invoke("Hello, world!")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--8dda4598-a5cd-4308-9a84-3368b5629211-0', usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}})

# Promt Templates
Define reusable prompt templates that act as structured instructions for the LLM. This makes input formatting consistent and easier to manage.

In [9]:
from langchain.prompts import PromptTemplate

In [4]:
from langchain.prompts import PromptTemplate

summary_template = PromptTemplate(
    input_variables=["document_text"],
    template="""
You are a professional document summarizer.

Your task is to analyze the following block of text extracted from a PDF document. First, briefly infer what kind of document this is (e.g., legal contract, academic paper, invoice, manual, report, etc.). Then generate a clear and concise summary in bullet points.

Instructions:
- Start by stating what type of document it appears to be, in one short sentence.
- Then summarize the key points in bullet format (max 150 words total).
- Focus on the most relevant facts or ideas.
- Avoid copying entire sentences from the original.
- Use neutral, objective language.

Text to analyze:
{document_text}

Output:
"""
)

In [ ]:
qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an intelligent assistant. You are given a block of text extracted from a PDF document.
Use only the information provided in the context below to answer the user's question as clearly and accurately as possible.

If the answer is not explicitly mentioned in the context, respond with "The answer is not available in the document."

Context:
{context}

Question:
{question}

Answer:
"""
)

# Create the Langchain Chain
Build a chain that combines the prompt and the LLM into a single callable object. This lets us run the model in a pipeline-friendly way.

In [5]:
from langchain.chains import LLMChain

In [10]:
chain = summary_template | model

# Testing out the chain
Pass sample text into the chain to verify that the prompt and LLM integration work correctly. This step confirms that the summarization or other logic behaves as expected.

In [11]:
text = """
Technology has become an essential part of our daily lives. From the moment we wake up to the time we go to sleep, we interact with some form of technology. Smartphones, computers, and smart devices help us communicate, work, study, and entertain ourselves more efficiently than ever before.

In the past, simple tasks like sending a letter or finding a recipe could take hours or even days. Today, these things can be done instantly with a few clicks or voice commands. Education has also changed — students can attend classes online, access global libraries, and use AI tools to improve their learning.

However, technology also brings challenges. People spend more time in front of screens, which can affect their health and reduce face-to-face interactions. Additionally, rapid technological change means that workers need to constantly update their skills to remain relevant.

In conclusion, technology offers many benefits, but it also requires us to adapt and use it responsibly.
"""

In [12]:
result = chain.invoke({'document_text':text})

In [13]:
result

AIMessage(content='This appears to be an excerpt from a general overview or introduction to a discussion about technology.\n\n*   Technology is integral to modern life, facilitating communication, work, study, and entertainment.\n*   Technology enhances efficiency, enabling instant completion of tasks that previously took significant time.\n*   Education has been transformed through online classes, access to global resources, and AI tools.\n*   Increased screen time poses health risks and reduces in-person social interaction.\n*   Rapid technological advancements necessitate continuous upskilling for workers.\n*   Technology offers benefits but requires responsible and adaptive use.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--78c3c4c5-60eb-4697-bc71-e5a8e79a58c9-0', usage_metadata={'input_tokens': 335, 'output_tokens': 120, 'total_token